# Experimental Model for a Forest Fire Observation Drone

Copyright 2023 MBSE4U


Since it is an experimental model, it is neither complete nor correct. Be inspired and feel free to provide feedback tim@mbse4u.com.



In [26]:
package ForestFireObservationDrone {
    
    import Drone_SharedAssetsSuperset::**;
    
    part forestFireObservationDrone : Drone {
        
        satisfy Drone_FeatureModel::droneFeatureConfigurations.forestFireObserverationDrone by self;

        //
        // Variant selections
        //
        
        // numberOfEngines
        :>> numberOfEnginesVariation = numberOfEnginesVariation::sixEngines;
        :>> body = DroneBodyVariation::droneBody6Engines {
            :>> bodyColor = bodyColor.redColor;
        }
        :>> flightControl = FlightControlVariation::droneFlightControl4Engines;
        
        // Battery
        :>> battery = DroneBatteryVariation::longDistanceBattery;       

        // safetyFeatures
        // autonomousFlight
        
        // Engine
        part engine1 :> engines {
            :>> propKind = PropKind::clockwise;
        }
        part engine2 :> engines {
            :>> propKind = PropKind::counterclockwise;
        }
        part engine3 :> engines {
            :>> propKind = PropKind::clockwise;
        }        
        part engine4 :> engines {
            :>> propKind = PropKind::counterclockwise;
        }
        
        // Connections
        connect powerManagementModule to engine1;
        connect powerManagementModule to engine2;
        connect powerManagementModule to engine3;
        connect powerManagementModule to engine4;                    
    }
}

Package ForestFireObservationDrone (a3d29308-bb71-4b73-ad2c-6713f23843d6)


In [ ]:
%viz ForestFireObservationDrone

# Feature Model

In [5]:
package Drone_FeatureModel {
    
    requirement <'FD1'> droneFeatures : Feature {
        
        doc /* Root feature of the drone product line */
                    
        abstract requirement numberOfEngines : Feature {
            doc /* The drone product line offers drones with four or with six engines. */
            :>> bindingTime = BindingTimeKind::Design;
        }
        variation requirement <'FD1.1'> numberOfEnginesVariation :> numberOfEngines {
            variant requirement fourEngines : Feature {
                doc /* A drone with four engines. */
            }
            variant requirement sixEngines : Feature {
                doc /* A drone with six engines. */
                require batteryKindVariation::longDistanceBattery;
            }
        }
        
        abstract requirement batteryKind : Feature {
            doc /* The drone product line offers standard batteries and more powerful batteries for long distance flights. */
            :>> bindingTime = BindingTimeKind::Operation;                
        }
        variation requirement <'FD1.2'> batteryKindVariation :> batteryKind {
            variant requirement standardBattery : Feature {
                doc /* Standard battery with a capacity of at least 3000 mAh. */
            }
            variant requirement longDistanceBattery : Feature {
                doc /* Battery for long distance flights with a capacity of at least 5000 mAh. */
            }
        }
        
        abstract requirement safetyFeatures : Feature[0..3] {
            :>> bindingTime = BindingTimeKind::Design;
        }
        variation requirement <'FD1.3'> safetyFeaturesVariation :> safetyFeatures {
            variant requirement geoFencing : Feature;
            variant requirement returnHome : Feature;
            variant requirement collisionAvoidance : Feature;
        }

        abstract requirement autonomousFlight : Feature[0..3]  {
            :>> bindingTime = BindingTimeKind::Design;
        }
        variation requirement <'FD1.4'> autonomousFlightVariation :> autonomousFlight  {
            variant requirement collisionAvoidance : Feature;
            variant requirement flightPathDefinition : Feature;
            variant requirement flightDestinationDefinition : Feature;
        }

        abstract requirement flightPatterns : Feature {
            :>> bindingTime = BindingTimeKind::Production;
        }
        variation requirement <'FD1.5'> flightPatternsVariation :> flightPatterns {
            variant requirement flyObservationPatterns : Feature;
            variant requirement flyCameraPatterns : Feature;
            variant requirement flyFunPatterns : Feature;
        }

        abstract requirement bodyColor : Feature {
            :>> bindingTime = BindingTimeKind::Production;
        }        
        variation requirement <'FD1.6'> bodyColorVariation : Feature {
            variant requirement redBody : Feature;
            variant requirement whiteBody : Feature;
            variant requirement blueBody : Feature;
            variant requirement blackBody : Feature;  
        }
    }
    
    requirement <'FC'> droneFeatureConfigurations : Feature {
        
        //
        // Feature Configurations
        //
        requirement <'FC1'> standardObserverationDrone :> droneFeatures {
            doc /* A standard drone with basic capabilities. */
            :>> numberOfEnginesVariation = numberOfEnginesVariation::fourEngines;
            :>> batteryKindVariation = batteryKindVariation::standardBattery;
            :>> safetyFeatures = ();
            :>> autonomousFlight = (autonomousFlightVariation::flightPathDefinition);
            :>> flightPatterns = flightPatternsVariation::flyObservationPatterns;
            :>> bodyColor = bodyColorVariation::blueBody;
        }

        requirement <'FC2'> forestFireObserverationDrone :> droneFeatures {
            doc /* A drone with special capabilities to detect forest fires. */
            :>> numberOfEnginesVariation = numberOfEnginesVariation::sixEngines;
            :>> batteryKindVariation = batteryKindVariation::longDistanceBattery;
            :>> safetyFeatures = (safetyFeaturesVariation::collisionAvoidance, 
                                  safetyFeaturesVariation::returnHome);
            :>> autonomousFlight = (autonomousFlightVariation::collisionAvoidance,
                                    autonomousFlightVariation::flightPathDefinition,
                                    autonomousFlightVariation::flightDestinationDefinition);
            :>> flightPatterns = flightPatternsVariation::flyObservationPatterns;
            :>> bodyColor = bodyColorVariation::redBody;
        }     
    }
    
    requirement def Feature {
        attribute bindingTime : BindingTimeKind;
    }
    enum def BindingTimeKind {
        enum Design;
        enum Production;
        enum Operation;
    }    
}


Package Drone_FeatureModel (81fee6a8-e60d-46bf-8f45-13bac0d16f32)


In [ ]:
package Drone_FeatureModel {
    
    import PLEML::*;
    
    #feature droneFeatures {    
        #requiredFeature requirement numberOfEngines {
            #feature fourEngines[0..1];
            #feature sixEngines [0..1] {
                #requiredFeature rf1 = batteryKind.longDistanceBattery;
            }
            require constraint {
                (fourEngines == null) xor (sixEngines == null)
            }                
        }
        
        #requiredFeature batteryKind {
            #feature standardBattery;
            #feature longDistanceBattery;
        }
        
        #requiredFeature flightPatterns {
            #feature flyObservationPatterns;
            #feature flyCameraPatterns;
            #feature flyFunPatterns;
        }
    } 

    #featureConfiguration forestFireObservationDrone :> droneFeatures {
        :>> droneFeatures.numberOfEngines.sixEngines[1];
        :>> droneFeatures.batteryKind.longDistanceBattery[1];
        :>> droneFeatures.flightPatterns.flyObservationPatterns[1];
    }
    
    view DroneFeatureTree {
        expose droneFeatures::**;
        render Views::asTreeDiagram;
    }
}

# Shared Assets Superset

The shared asset superset contains all shared assets of the drone product line. The forest fire observation drone is one product of the product line. Other member products could be a camera drone, a surveillance drone, and so forth.

In [23]:
library package Drone_SharedAssetsSuperset {
    
    package Drone {
               
        import DroneBattery::**;
        import DroneEngine::**;
        import DroneBody::**;
        import DroneFlightControl::**;  
        
        abstract part def Drone {
            
            // Attributes
            attribute massActual : ISQ::MassValue;
            
            
            // Spatial extent
            item :>> shape = body.shape;
            item boundingBox :> boundingShapes = body.boundingBox;
            
            // Timeslices            
            timeslice parking {
                snapshot startCharging {
                    // model some start charging conditions
                }
                then snapshot stopCharging;
            }
            then timeslice flying {
                :>> battery[1];
            }
            
            // States
            state droneStates {
                entry; then off;

                state off;
                accept SigSwitchOn then standBy;
                state standBy;
                accept SigSwitchOff then off;
                transition standBy_charging
                    first standBy
                    accept SigStartCharging
                    then charging;
                transition standBy_ready 
                    first standBy
                    accept SigActivate
                    then ready;
                state charging;
                    transition charging_standBy
                        first charging
                        accept SigStopCharging
                        then standBy;
                state ready;
                accept SigDeactivate then standBy;
                transition ready_flying
                    first ready
                    accept SigStartFlying 
                    then flying;
                
                state flying;    
                accept SigStopFlying then ready;
            }
            
            // Signals
            attribute def SigStartCharging;
            attribute def SigStopCharging;
            attribute def SigSwitchOn;
            attribute def SigSwitchOff;
            attribute def SigActivate;
            attribute def SigDeactivate;
            attribute def SigStartFlying;
            attribute def SigStopFlying;
            
            // Power
            part battery : DroneBatteryVariation;
            message notifyBatteryLow of SigBatteryLow from battery.batteryManagementSystem to flightControl;
            
            part charger;
            part powerManagementModule {
                port ctrlPort {
                    out attribute status : BatteryCapacityKind;
                }
            
                perform action checkStatus {
                    out attribute status : BatteryCapacityKind;
                
                    first start;
                    then action getBatteryStatus;
                    then action analyseStatus;
                    then done;
                } 
            }
            
            // Engines
            variation attribute numberOfEnginesVariation : ScalarValues::Integer {
                variant attribute fourEngines = 4;
                variant attribute sixEngines = 6;
            }
            assert constraint { 
                ((numberOfEnginesVariation == numberOfEnginesVariation::sixEngines) and 
                (battery == DroneBattery::DroneBattery_Parts::DroneBatteryVariation::longDistanceBattery) and
                (body == DroneBody::DroneBody_Parts::DroneBodyVariation::droneBody6Engines) and
                (flightControl == DroneFlightControl::FlightControlVariation::droneFlightControl4Engines)) or
                ((numberOfEnginesVariation == numberOfEnginesVariation::fourEngines) and 
                (body == DroneBody::DroneBody_Parts::DroneBodyVariation::droneBody4Engines) and
                (flightControl == DroneFlightControl::FlightControlVariation::droneFlightControl4Engines))
            }            
            satisfy Drone_FeatureModel::droneFeatures.numberOfEngines by numberOfEnginesVariation;                
            part engines[numberOfEnginesVariation] : DroneEngine;
            
            // Body
            part body : DroneBodyVariation;
            
            // Flight Control System
            part flightControl : FlightControlVariation;
            part droneControlUnit {
                port powerCtrlPort {
                    in attribute batteryStatus : BatteryCapacityKind;
                }
            }
            
            part safetyModule[*];
            part autonomousFlightModules[*];
            :>> safetyFeatures = (safetyFeaturesVariation::collisionAvoidance, 
                                  safetyFeaturesVariation::returnHome);
            :>> autonomousFlight = (autonomousFlightVariation::collisionAvoidance,
                                    autonomousFlightVariation::flightPathDefinition,
                                    autonomousFlightVariation::flightDestinationDefinition);
            
            
            
            part gpsModule;
            part inertialMeasurementUnit;
            part barometer;         
            
            // Communication System
            part radioTransmitter;
            part radioReceiver;
            part antenna;
        
            // Sensors
            part camera;
            part obstacleAvoidanceSensor;
            
            // Connections
            connect droneControlUnit.powerCtrlPort to powerManagementModule.ctrlPort;
            connect battery to powerManagementModule;
        }

        //
        // Drone Requirements
        //
        package Drone_Requirements {
        
            requirement droneStakeholderRequirements {
                subject drone : Drone;
                
                require droneEngineStakeholderRequirements;
                require droneBatteryStakeholderRequirements;
                
                variation requirement maximumTakeoffWeight {
                    variant requirement mtqw_Quadcopter : MTOW_Quadcopter;
                    variant requirement mtqw_Hexacopter : MTOW_Hexacopter;
                }
            }

            requirement def <'D-REQ-22'> MTOW {
                
                subject drone : Drone;
                
                doc /*
                    * The maximum takeoff weight (MTOW) for a drone shall not exceed [X] kilograms. 
                    * This includes the weight of the drone itself, its battery, payload, and any additional 
                    * attached equipment. [X] depends on the number of engines.
                    */
                    
                attribute mtow : ISQ::MassValue;
                require constraint { drone.massActual <= mtow }
            }
            
            
            requirement def <'D-REQ-23'> MTOW_Quadcopter :> MTOW {
                doc /*
                    * The maximum takeoff weight (MTOW) for a quadcopter equipped with 4 engines shall not exceed 
                    * 15 kilograms. 
                    */
                :>> mtow = 15[SI::kg];
            }

            requirement def <'D-REQ-24'> MTOW_Hexacopter :> MTOW {
                doc /*
                    * The maximum takeoff weight (MTOW) for a hexacopter equipped with 6 engines shall not exceed 
                    * 25 kilograms. 
                    */
                :>> mtow = 25[SI::kg];
            }
        }
        
        //
        // Drone Use Cases
        //
        package Drone_UseCases {
            use case observeArea {
                subject drone : Drone;
                
                in item hi_observationArea;
            
                first start;
                then action defineObservationArea {
                    in item hi_observationArea = observeArea::hi_observationArea;
                    out item observationArea;
                }
                then action approachArea {
                    in item observationArea;
                }
                then action flyPatterns : FlyPatterns;
                then action returnToHomeBase;
                then done;            
                flow defineObservationArea.observationArea to approachArea.observationArea;
            }
        
            use case chargeDrone {
                subject drone : Drone;
            
                first start;
                then action plugDrone;
                then action chargeDroneBatteries;
                then done;
            }
            
            //
            // Action Definitions
            //
            variation action def FlyPatterns {
                variant action flyObservationPatterns {
                    satisfy Drone_FeatureModel::droneFeatures.flightPatternsVariation.flyObservationPatterns by self;
                }
                variant action flyCameraPatterns {
                    satisfy Drone_FeatureModel::droneFeatures.flightPatternsVariation.flyCameraPatterns by self;
                }
                variant action flyFunPatterns {
                    satisfy Drone_FeatureModel::droneFeatures.flightPatternsVariation.flyFunPatterns by self;
                }
            }   
        }
    }
     
    //
    // Drone Engine Shared Assets
    //
    package DroneEngine {
        
        package DroneEngine_StakeholderRequirements {
            
            import DroneEngine_Parts::*;
            
            variation requirement droneEngineStakeholderRequirements {
                variant requirement droneEngineStandardStakeholderRequirements {
                    
                    subject engine : DroneEngine;
                                        
                    requirement power : Power;
                    requirement efficiency : Efficiency;
                    requirement durability : Durability;
                    requirement reliability : Reliability;
                    requirement lightweight : Lightweight;
                    requirement control : Control;
                    requirement compatibility : Compatibility;
                    requirement safety : Safety;
                    requirement costEffective : CostEffective;
                }
                variant requirement droneEngineLowNoiseStakeholderRequirements :> droneEngineStandardStakeholderRequirements {
                    requirement lowNoise : LowNoise;
                }
            }
            
            requirement def <'DE-REQ-1'> Power {
                doc /*
                    * The engine shall provide enough power to lift the drone off the ground and keep it in the air. 
                    * The power output shall be appropriate for the size and weight of the drone.
                    */
            }
            requirement def <'DE-REQ-2'> Efficiency {
                doc /*
                    * The engine shall be efficient, allowing the drone to fly for an extended period on a single charge. 
                    * The engine shall be designed to minimize energy loss through friction and heat.
                    */
            }
            requirement def <'DE-REQ-3'> Durability {
                doc /*
                    * The engine shall be able to withstand the stress of continuous use and exposure to the elements. 
                    * It shall be able to operate in a wide range of temperatures and weather conditions.
                    */
            }
            requirement def <'DE-REQ-4'> Reliability {
                doc /*
                    * The engine shall be reliable, with minimal downtime or maintenance requirements. 
                    * It shall be designed with high-quality components and undergo rigorous testing before deployment.
                    */
            }
            requirement def <'DE-REQ-5'> LowNoise {
                doc /*
                    * The engine shall produce minimal noise to avoid disturbing the environment or attracting unwanted attention.
                    */
            }
            requirement def <'DE-REQ-6'> Lightweight {
                doc /*
                    * The engine shall be lightweight, allowing for maximum payload capacity and longer flight times.
                    */
            }
            requirement def <'DE-REQ-7'> Control {
                doc /*
                    * The engine shall be easy to control, allowing for precise maneuverability and responsiveness to commands.
                    */
            }
            requirement def <'DE-REQ-8'> Compatibility {
                doc /*
                    * The engine shall be compatible with the drone's power supply and electronic components.
                    */
            }
            requirement def <'DE-REQ-9'> Safety {
                doc /*
                    * The engine should be designed with safety in mind, minimizing the risk of injury or damage in case of malfunction. 
                    * It should have fail-safe mechanisms to prevent accidents and minimize damage in case of emergency.
                    */
            }
            requirement def <'DE-REQ-10'> CostEffective {
                doc /*
                    * The engine shall be cost-effective, balancing performance and quality with affordability. 
                    * It shall provide value for money, with a low cost of ownership and maintenance.            
                    */
            }            
        }
        
        package DroneEngine_Parts {
            part def DroneEngine {
                enum propKind : PropKind;
            
                enum def PropKind {
                    enum clockwise;
                    enum counterclockwise;
                }            
            }
        }                        
    }
    
    
    //
    // Drone Body Shared Assets
    //
    package DroneBody {
        
        package DroneBody_Parts {
            
            import DroneBody_Domain::*;

            variation part def DroneBodyVariation :> DroneBody {
                variant part droneBody4Engines : DroneBody {
                    // to be done: redefines size
                    :>> rotors[4];
                    :>> arms[4];
                    satisfy Drone_FeatureModel::droneFeatures.numberOfEnginesVariation.fourEngines by self;
                }
                variant part droneBody6Engines : DroneBody {
                    // to be done: redefines size
                    :>> rotors[6];
                    :>> arms[6];
                    satisfy Drone_FeatureModel::droneFeatures.numberOfEnginesVariation.sixEngines by self;
                }
            }            
            
            abstract part def DroneBody {
                // Spatial extent
                item :>> shape : ShapeItems::Cylinder {
                    :>> height = 20 [SI::cm];
                    :>> base.shape : ShapeItems::Circle {
                        :>> radius = 60 [SI::cm];
                    }
                }
                item boundingBox : ShapeItems::Box [1] :> boundingShapes {
                    :>> length = 120 [SI::cm];
                    :>> width  = 120 [SI::cm];
                    :>> height = 20 [SI::cm];
                }            
                
                // BodyColorKind is an enumeration definition which is already a kind of variation
                attribute bodyColor : BodyColorKind {
                    enum redColor redefines red {
                        satisfy Drone_FeatureModel::droneFeatures.bodyColorVariation.redBody by self;
                    }
                    enum whiteColor redefines white {
                        satisfy Drone_FeatureModel::droneFeatures.bodyColorVariation.whiteBody by self;
                    }
                    enum blueColor redefines blue {
                        satisfy Drone_FeatureModel::droneFeatures.bodyColorVariation.blueBody by self;
                    }
                    enum blackColor redefines black {
                        satisfy Drone_FeatureModel::droneFeatures.bodyColorVariation.blackBody by self;
                    }
                }    
                                
                part rotors[*];
                part arms[*];
                part landingGear;
            }
        }
        
        package DroneBody_Domain {            
            enum def BodyColorKind {
                enum red;
                enum white;
                enum blue;
                enum black;
            }
        }
    }
    
    
    //
    // Drone Flight Control Software Shared Assets
    //
    package DroneFlightControl {
        
        variation part def FlightControlVariation :> DroneFlightControl {
            variant part droneFlightControl4Engines : DroneFlightControl {
                satisfy Drone_FeatureModel::droneFeatures.numberOfEnginesVariation.fourEngines by self;
            }
            variant part droneFlightControl6Engines : DroneFlightControl {
                satisfy Drone_FeatureModel::droneFeatures.numberOfEnginesVariation.sixEngines by self;
            }
        }
        abstract part def DroneFlightControl;
    }

    //
    // Drone Battery Shared Assets
    //     
    package DroneBattery {
        
        package DroneBattery_Domain {
            attribute def SigBatteryLow {
                attribute capacity : ISQElectromagnetism::ElectricChargeValue;
            }
        }
        
        package DroneBattery_Parts {

            variation part def DroneBatteryVariation :> DroneBattery {
                variant part standardBattery : StandardDroneBattery {
                    satisfy Drone_FeatureModel::droneFeatures.batteryKindVariation.standardBattery by self;
                }
                variant part longDistanceBattery : LongDistanceDroneBattery {
                    satisfy Drone_FeatureModel::droneFeatures.batteryKindVariation.longDistanceBattery by self;
                }
            }            
            
            abstract part def DroneBattery {
                // Attributes
                attribute weight : ISQBase::MassValue; 
                attribute maxCapacity : ISQElectromagnetism::ElectricChargeValue;
                enum currentCapacityLevel : BatteryCapacityKind;
                enum def BatteryCapacityKind :> ScalarValues::Integer {
                    enum full = 100;
                    enum medium = 50;
                    enum low = 30;
                    enum critical = 15;
                }
            
                // Parts
                part batteryCells {
                    doc /* The battery cells are the main components of the drone battery 
                         * that store the electrical energy. These cells are usually Lithium Polymer (LiPo) 
                         * or Lithium-Ion (Li-Ion) cells.
                        */
                }
                part batteryManagementSystem {
                    doc /* The Battery Management System (BMS) is a control unit that manages the charging and 
                         * discharging of the battery cells. It ensures that the cells are charged and discharged 
                         * correctly and safely.
                         */
                }
                part protectionCircuit {
                    doc /* The protection circuit is a safety mechanism that protects the battery from overcharging, 
                         * over-discharging, and short circuits.
                        */
                }
                part powerConnector {
                    doc /* The power connector is the interface that connects the battery to the drone's power system. 
                        * It delivers the electrical energy from the battery to the drone.
                        */
                }
                part chargingInterface {
                    doc /* The charging interface is the interface that connects the battery to the charger. 
                         * It allows the battery to be charged safely and efficiently.
                        */
                }
                part batteryIndicator {
                    doc /* The battery indicator is a component that displays the remaining battery charge. 
                         * It allows the pilot to monitor the battery level and plan flight time accordingly.
                        */
                }
                part communicationInterface {
                    doc /* The communication interface is a component that allows the battery to communicate 
                         * with the drone's flight controller. It provides important battery information such 
                         * as voltage, current, and temperature to the flight controller.
                        */
                    }
                part firmware {
                    doc /* The firmware is the software that runs on the battery management system. 
                         * It controls the charging and discharging of the battery cells and ensures 
                         * that the battery operates safely and efficiently.
                        */
                }
            }
                             
            part def StandardDroneBattery :> DroneBattery {
                :>> weight = 275[SI::g];
                :>> maxCapacity = 13680[SI::'s⋅A'];
            }
            part def LongDistanceDroneBattery :> DroneBattery {
                :>> weight = 315[SI::g];
                :>> maxCapacity = 18000[SI::'s⋅A'];
            }
        }
            
        package DroneBattery_SystemRequirements {
            
            import RequirementDerivation::*;
            import DroneBattery_StakeholderRequirements::*;
            import DroneBattery_Parts::*;
            
            variation requirement droneBatterySystemRequirements {
                variant requirement droneBatteryStandardSystemRequirements {
                    satisfy Drone_FeatureModel::droneFeatures.batteryKindVariation.standardBattery by self;
                    subject battery : DroneBattery;    
                }
                variant requirement droneBatteryLongDistanceSystemRequirements {
                    satisfy Drone_FeatureModel::droneFeatures.batteryKindVariation.longDistanceBattery by self;
                    subject battery : DroneBattery;
                    
                    requirement longDistanceCapacity : LongDistanceCapacity;
                    #derivation connection {
                        end #original ::> droneBatteryStakeholderRequirements.highCapacity;
                        end #derive ::> longDistanceCapacity;
                    }
                }                                               
            }
            satisfy Drone_FeatureModel::droneFeatures.batteryKind by droneBatterySystemRequirements;
            
            requirement def <'DB-REQ-421'> LongDistanceCapacity {
                doc /* The battery shall have a capacity of at least 13860 sA, which provides enough power for extended flight times. */
                    
                subject battery : DroneBattery;
                
                assert constraint {
                    (battery.maxCapacity >= 13860[SI::'s⋅A'])
                }
            }
        }
        
        package DroneBattery_StakeholderRequirements {
            
            import DroneBattery_Parts::*;
            
            requirement droneBatteryStakeholderRequirements {
                
                subject battery : DroneBattery;
                
                requirement lightweight : Lightweight;
                requirement highCapacity : HighCapacity;
                requirement fastCharging : FastCharging;
                requirement longLife : LongLife;
                requirement safe : Safe;
                requirement compatible : Compatible;
                requirement reliable : Reliable;
                requirement affordable : Affordable;
                requirement easyToInstall : EasyToInstall;
            }
            
            requirement def <'DB-REQ-2'> Lightweight {
                doc /*
                     * A drone battery shall be lightweight to keep the overall weight of the drone 
                     * to a minimum, which is essential for stable flight.
                    */
            }

            requirement def <'DB-REQ-3'> HighCapacity {
                doc /*
                     * A drone battery shall have a high capacity to provide the drone with enough power 
                     * to fly for a reasonable amount of time.
                    */
            }

            requirement def <'DB-REQ-4'> FastCharging {
                doc /*
                     * A drone battery shall be able to charge quickly so that the drone can 
                     * get back in the air as soon as possible.
                    */
            }

            requirement def <'DB-REQ-5'> LongLife {
                doc /*
                     * A drone battery shall have a long life to avoid the need for frequent replacements.
                    */
            }

            requirement def <'DB-REQ-6'> Safe {
                doc /*
                     * A drone battery shall be designed to be safe, with features such as overcharge protection, 
                     * short circuit protection, and temperature control.
                    */
            }

            requirement def <'DB-REQ-7'> Compatible {
                doc /*
                     * A drone battery shall be compatible with the drone's power system to ensure proper functioning.
                    */
            }
            
            requirement def <'DB-REQ-8'> Reliable {
                doc /*
                     * A drone battery shall be reliable, with consistent performance and the ability to withstand 
                     * the stresses of flight and charging cycles.
                    */
            }

            requirement def <'DB-REQ-9'> Affordable {
                doc /*
                     * A drone battery shall be reasonably priced, to avoid being a significant expense for drone pilots.
                    */
            }
            
            requirement def <'DB-REQ-10'> EasyToInstall {
                doc /*
                     * A drone battery shall be easy to install and remove to make it easy for pilots to swap out batteries when necessary.
                    */
            }
        }
    }    
}


LibraryPackage Drone_SharedAssetsSuperset (772e8417-f390-4c99-882d-df5ad6dd6c7f)


In [ ]:
%viz --view=state Drone_SharedAssetSuperset::DroneProductLine::Drone::droneStates

In [ ]:
%viz --view=state Drone_SharedAssetSuperset::DroneProductLine::Drone::droneStates2

In [ ]:
%show Drone_SharedAssetsSuperset::DroneProductLine::Drone

In [ ]:
%publish Drone_FeatureModel

In [ ]:
%show Drone_FeatureModel

# PLEML - SysML Extension for MBPLE

The MBPLE extension Product Line Engineering Modeling Language (PLEML) provides concepts for modeling features with SysML v2.

In [ ]:
library package PLEML {
    
    doc /* 
        *  Product Line Engineering Modeling Language (PLEML) 
        *  Language extension for SysML 
        */
        
    // Concepts
    requirement def Feature {
        doc /* A Feature is based on a SysML requirement. */
        attribute bindingTime : BindingTimeKind = BindingTimeKind::Design;
        
        ref requirement :>> requiredFeatures;
        require constraint {
            requiredFeatures->ControlFunctions::forAll{in r; r}
        } 
    }
    requirement features[*] : Feature;

    enum def BindingTimeKind {
        enum Design;
        enum Production;
        enum Operation;
    }
    
    ref requirement requiredFeatures[*] : Feature;
    
    requirement def FeatureConfiguration {
        attribute lifeCycle : BindingTimeKind = BindingTimeKind::Production;
    }
    requirement featureConfigurations[*] : FeatureConfiguration;
    
    // Domain-specific Metadata
    metadata def <feature> FeatureDSM :> Metaobjects::SemanticMetadata {
        :>> baseType = features meta SysML::RequirementUsage;
    }
    metadata def <requiredFeature> RequiredFeatureDSM :> Metaobjects::SemanticMetadata {
        :>> baseType = requiredFeatures meta SysML::RequirementUsage;
    }
    metadata def <featureConfiguration> FeatureConfigurationDSM :> Metaobjects::SemanticMetadata {
        :>> baseType = featureConfigurations meta SysML::RequirementUsage;
    }    
}